In [26]:
!pip install requests
!pip install beautifulsoup4
!pip install --upgrade geopandas


In [27]:
import requests
from bs4 import BeautifulSoup
import re
import zipfile
import io
import os
from datetime import datetime
import shutil
import geopandas as gpd
import pandas as pd
import rasterio 

# Define the arguments directly
args = {
    'local_folder': './sat/data',
    'directory': './sat/outputs',
    'fire_fp': r'C:\Users\mnlee2\Desktop\data\saved_outputs\sobol_inputs_2\inputs=inputs_WIND=ORIGINAL_SPOTTING=.TRUE._S-CONFIG=799-lhs-large_CROWN-FIRE-MODEL=0\time_of_arrival_0000001_0347428.tif',
    'start_date': '2020-09-05',
    'end_date': '2020-09-12',
    'url': 'https://ftp.wildfire.gov/public/incident_specific_data/calif_s/2020_Incidents/CA-SNF-001391_Creek/IR/NIROPS/',
    'viirs_data_path': '/content/drive/My Drive/NASA/data/VIIRS/viirs_complete.csv'
}

# Simulate argparse functionality by accessing arguments directly
class ArgsNamespace:
    def __init__(self, args_dict):
        for key, value in args_dict.items():
            setattr(self, key, value)

args_namespace = ArgsNamespace(args)

# Create directories if they don't exist
if not os.path.exists(args_namespace.local_folder):
    os.makedirs(args_namespace.local_folder)

if not os.path.exists(args_namespace.directory):
    os.makedirs(args_namespace.directory)

#Define the variable names
local_folder = args_namespace.local_folder
directory = args_namespace.directory
fire_fp = args_namespace.fire_fp
start_date = datetime.strptime(args_namespace.start_date, '%Y-%m-%d')  # Convert to datetime
end_date = datetime.strptime(args_namespace.end_date, '%Y-%m-%d')      # Convert to datetime
url = args_namespace.url
viirs_data_path = args_namespace.viirs_data_path

AttributeError: module 'os' has no attribute 'add_dll_directory'

In [20]:
########################################
# Goes to website and collects all the nessecary shape files
# May have some minor bugs but works enough for rough analysis

# Send a GET request to the webpage
response = requests.get(url)

# Parse the webpage with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the links on the webpage
links = soup.find_all('a')

# Regular expression to match the folder names
pattern = re.compile(r'(\d{8}_OAR|^\d{8})/$')

##########################
print('Dates Found:')
# Iterate over the links to print the dates
for link in links:
    href = link.get('href')

    # If the link matches the pattern
    if pattern.match(href):
        # Extract the date from the folder name
        date_str = re.search(r'\d{8}', href).group()
        date = datetime.strptime(date_str, '%Y%m%d')

        # Print the date
        print(date.strftime('%Y-%m-%d'))
###########################
zip_found = False
# Iterate over the links
for link in links:
    href = link.get('href')

    # If the link matches the pattern
    if pattern.match(href):
        # Extract the date from the folder name
        date_str = re.search(r'\d{8}', href).group()
        date = datetime.strptime(date_str, '%Y%m%d')


        # Check if the date is within the range
        if start_date <= date <= end_date:
            # Print the date
            print('Current Date: ', date.strftime('%Y-%m-%d'))

            # The URL of the zip file
            zip_url = url + href

            # Send a GET request to the zip file URL
            zip_response = requests.get(zip_url)

            # Parse the webpage with BeautifulSoup
            zip_soup = BeautifulSoup(zip_response.text, 'html.parser')

            # Find all the links on the webpage
            zip_links = zip_soup.find_all('a')

            # Iterate over the links
            for zip_link in zip_links:
                zip_href = zip_link.get('href')

                # If there are multiple zip files in the folder
                if len(zip_links) > 1:
                    # Use the pattern that includes 'shape'
                    zip_pattern = re.compile(r'.*shape.*\.zip$', re.IGNORECASE)
                else:
                    # Use the pattern that matches any zip file
                    zip_pattern = re.compile(r'.*\.zip$')

                # If the link matches the pattern
                if zip_pattern.match(zip_href):
                    # Set the flag to True
                    zip_found = True

                    # The URL of the zip file
                    file_url = zip_url + zip_href
                    print(file_url)

                    # Send a GET request to the file URL
                    file_response = requests.get(file_url)

                    # Check if the request was successful
                    if file_response.status_code == 200:
                        # Open the zip file
                        with zipfile.ZipFile(io.BytesIO(file_response.content)) as z:
                            # Extract only the files that contain 'HeatPerimeter' or 'BurnPerimeter' in their names
                            print('Files Collected: ')
                            for file in z.namelist():
                                if 'Perimeter' in file or 'BurnPerimeter' in file:
                                    z.extract(file, path=os.path.join(local_folder, date.strftime('%Y-%m-%d')))
                    else:
                        print(f"Error: Failed to download {file_url}")

            # If no zip file was found, print a message
            if not zip_found:
                print(f"No zip file found for date {date.strftime('%Y-%m-%d')}")

#############################################################################################################3
# Clean up sub-folders and prepare for processing

# The path to the data folder
data_path = os.path.join(directory, local_folder)
data_path = local_folder
# Iterate over the dated subfolders
for date_folder in os.listdir(data_path):
    date_folder_path = os.path.join(data_path, date_folder)

    # If the path is a directory
    if os.path.isdir(date_folder_path):
        # Iterate over the subfolders
        for subfolder in os.listdir(date_folder_path):
            subfolder_path = os.path.join(date_folder_path, subfolder)

            # If the path is a directory
            if os.path.isdir(subfolder_path):
                # Iterate over the files in the subfolder
                for file in os.listdir(subfolder_path):
                    file_path = os.path.join(subfolder_path, file)

                    try:
                        # Move the file to the dated parent folder
                        shutil.move(file_path, date_folder_path)
                    except shutil.Error as e:
                        # Handle the exception
                        print(f"Error moving file {file}: {e}")

                # Remove the subfolder
                shutil.rmtree(subfolder_path)


Dates Found:
2020-09-06
2020-09-08
2020-09-09
2020-09-10
2020-09-12
2020-09-13
2020-09-14
2020-09-15
2020-09-17
2020-09-20
2020-09-21
2020-09-22
2020-09-23
2020-09-24
2020-09-26
2020-09-27
2020-09-29
2020-10-01
2020-10-02
2020-10-03
2020-10-04
2020-10-05
2020-10-06
2020-10-07
2020-10-08
2020-10-09
2020-10-10
2020-10-11
2020-10-13
2020-10-15
2020-10-16
2020-10-17
2020-10-18
2020-10-19
2020-10-20
2020-10-22
2020-10-24
2020-10-25
2020-10-26
2020-10-27
2020-10-28
2020-10-29
2020-10-30
2020-10-31
2020-11-01
2020-11-02
2020-11-03
2020-11-04
2020-11-05
2020-11-06
2020-11-08
2020-11-10
2020-11-13
Current Date:  2020-09-06
No zip file found for date 2020-09-06
Current Date:  2020-09-08
https://ftp.wildfire.gov/public/incident_specific_data/calif_s/2020_Incidents/CA-SNF-001391_Creek/IR/NIROPS/20200908_OAR/20200908_Creek_IR_shapefiles.zip
Files Collected: 
Current Date:  2020-09-09
https://ftp.wildfire.gov/public/incident_specific_data/calif_s/2020_Incidents/CA-SNF-001391_Creek/IR/NIROPS/20200909

In [3]:
## Derive the Bounds for the Plots From the Last NIROPS Data of Interest
# Get the list of dated subfolders
date_folders = [folder for folder in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, folder))]

# Parse the dates and find the most recent one
ed = max(datetime.strptime(date, '%Y-%m-%d') for date in date_folders).strftime('%Y-%m-%d')
print('last date: ', ed)
date_folder_path = os.path.join(data_path, ed)

# Iterate over the files in the dated subfolder
for file in os.listdir(date_folder_path):
    # If the file is a shapefile
    if file.endswith('.shp'):
        file_path = os.path.join(date_folder_path, file)

        # Load the shapefile
        data = gpd.read_file(file_path)

# Assuming `data` is your GeoDataFrame and `geometry` is your geometry column
bounds = data.geometry.total_bounds

# This will give you a tuple in the format (minx, miny, maxx, maxy)
# which corresponds to (min longitude, min latitude, max longitude, max latitude)
print(bounds)
print(data.head())
# Plot the data
fig, ax = plt.subplots(1, 1)
data.plot(ax=ax, edgecolor='black')

# Add title and labels (optional)
ax.set_title('Heat Perimeter')

usage: ipykernel_launcher.py [-h] [--local_folder LOCAL_FOLDER]
                             [--directory DIRECTORY] --fire_fp FIRE_FP
                             --start_date START_DATE --end_date END_DATE --url
                             URL --viirs_data_path VIIRS_DATA_PATH
ipykernel_launcher.py: error: the following arguments are required: --start_date, --end_date, --url, --viirs_data_path


SystemExit: 2

c:\Users\mnlee2\AppData\Local\anaconda3\envs\elmfire_notebook\lib\site-packages\IPython\core\interactiveshell.py:3560: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#VIIRS Data Handling
# As CSV Files
# Read CSV file into a DataFrame
df = pd.read_csv(viirs_data_path)

# Assuming `df` is your DataFrame and `latitude` and `longitude` are your columns
#bounds = [-119.48449481, 36.98991853, -118.97099713, 37.64569761]
min_lon, min_lat, max_lon, max_lat = bounds #defined above

#Filter the dataframe on the appropriate ROI
#Don't Forget to Filter for Aquisition Date Too:

df = df[
        (df['longitude'] >= min_lon) &
        (df['longitude'] <= max_lon) &
        (df['latitude'] >= min_lat) &
        (df['latitude'] <= max_lat) &
        (df['frp'] > 1.0) #threshold for confidence
       ]


# Display the DataFrame
# All the dates are stored here:
print(df.shape)
#df.head()

In [ ]:
## COMBINED PLOT
#########
# This runs with the VIIRS points
from pyproj import Transformer
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from copy import copy

# The path to the data folder
data_path = local_folder

# Get the list of dated subfolders
date_folders = [folder for folder in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, folder))]

# Sort the dated subfolders in ascending order
date_folders.sort()

# Get the unique dates from the dataframe
unique_dates = df['acq_date'].sort_values().unique()

# Combine the dates from the dataframe and the subfolders
all_dates = np.unique(np.concatenate((date_folders, unique_dates)))

transformer = Transformer.from_crs("epsg:4326", "epsg:32611")

# start_date = datetime.strptime('2020-09-05', '%Y-%m-%d').date()
# end_date = datetime.strptime(ed, '%Y-%m-%d').date()
filtered_dates = [date for date in all_dates if start_date <= datetime.strptime(date, '%Y-%m-%d').date() <= end_date]

#filtered_dates = [date for date in all_dates if start_date <= datetime.strptime(date, '%Y-%m-%d') <= end_date]

# Calculate the number of columns needed for the subplots
n_columns = len(filtered_dates)

# Define the buffer size (in meters)
buffer_size = 5000

# Calculate the bounds of the asp data
xmin, ymin, xmax, ymax = src.bounds

# Apply the buffer to the bounds
xmin -= buffer_size
ymin -= buffer_size
xmax += buffer_size
ymax += buffer_size

# Open the fire perimeter file
with rasterio.open(fire_fp) as fire_src:
    # Read the fire perimeter data
    fire = fire_src.read(1)

    # Mask no_data values
    nodata = fire_src.nodata
    fire = np.ma.masked_equal(fire, nodata)


from datetime import datetime
def count_seconds(start_date, start_time, end_date, end_time):
    # Format for date and time
    format_str = '%Y-%m-%d %H%M'

    # Check if start_date and end_date are already datetime objects, if not convert them
    if isinstance(start_date, datetime):
        start = start_date
    else:
        # The time string should be padded with zeros if it is not four digits
        start = f"{start_date} {start_time.zfill(4)}"
        start = datetime.strptime(start, format_str)

    if isinstance(end_date, datetime):
        end = end_date
    else:
        # The time string should be padded with zeros if it is not four digits
        end = f"{end_date} {end_time.zfill(4)}"
        end = datetime.strptime(end, format_str)

    # Calculate the difference in seconds
    difference = end - start
    return difference.total_seconds()


# Create a grid of subplots
fig, axs = plt.subplots(2, n_columns, figsize=(20*n_columns, 16*3))

# Iterate over filtered dates
for column_idx, date in enumerate(filtered_dates):
    # Set column titles
    axs[0, column_idx].set_title(date, fontsize=45, pad=20)

    # Check if 'date' is a datetime object, if so format it as a string
    if isinstance(date, datetime):
        date = date.strftime('%Y-%m-%d')

    date_folder_path = os.path.join(data_path, date)

    # Plot the Aspect (if it should be plotted)
    #axs[column_idx].imshow(asp, cmap='viridis')

    # If the date exists in the VIIRS dataframe
    if date in unique_dates:
        # Filter the dataframe for the current date
        df_date = df[df['acq_date'] == date].copy()
        df_date.loc[:, 'x'], df_date.loc[:, 'y'] = transformer.transform(df_date['latitude'].values, df_date['longitude'].values)

        # Get the unique acquisition times
        unique_times = df_date['acq_time'].sort_values().unique()

        # For each unique time, plot in a separate subplot
        for row_idx, acq_time in enumerate(unique_times[:2]):

            # Filter the dataframe for the current acquisition time
            df_time = df_date[df_date['acq_time'] == acq_time]


            #scatter = axs[row_idx][column_idx].scatter(df_time['longitude'], df_time['latitude'], c=df_time['frp'], cmap='hot', alpha=0.5)
            asp_downsampled = asp[::10, ::10]  # take every 10th data point in each direction
            axs[row_idx][column_idx].imshow(asp_downsampled, cmap='gray', extent=(xmin, xmax, ymin, ymax))

            start_time = '0000'
            end_date = date
            end_time = str(copy(acq_time))
            print(start_date, start_time, end_date, end_time)
            max_val = count_seconds(start_date, start_time, end_date, end_time)
            print('Current Seconds: ', max_val )
            fire_masked = np.ma.masked_where(fire > max_val, fire)
            #axs[row_idx][column_idx].imshow(fire_masked, cmap='viridis', extent=(xmin, xmax, ymin, ymax))

            # Add the scatter points from the dataframe for the current acquisition time
            scatter = axs[row_idx][column_idx].scatter(df_time['x'], df_time['y'], c=df_time['frp'], cmap='hot', alpha=0.5, s=100)

            # Add a colorbar for the scatter plot
            #fig.colorbar(scatter, ax=axs[row_idx][column_idx], label='FRP')

            # Set the title for the subplot to the acquisition time
            axs[row_idx][column_idx].set_title(f"VIIRS Acquisition Time: {acq_time}", fontsize=36)

    # If the date exists in the subfolders
    if date in date_folders:
        # The path to the shapefile
        date_folder_path = os.path.join(data_path, date)

        # Iterate over the files in the dated subfolder
        for file in os.listdir(date_folder_path):
            # If the file is a shapefile
            if file.endswith('.shp'):
                file_path = os.path.join(date_folder_path, file)

                # Load the shapefile
                data = gpd.read_file(file_path)
                data = data.to_crs("EPSG:32611")

                # Plot the most recent NIROPS perimeter on all subplots for the current date
                for row_idx in range(2):
                    data.plot(ax=axs[row_idx][column_idx], edgecolor='black', alpha=0.4)

                    # # Set the limits of x-axis and y-axis
                    axs[row_idx][column_idx].set_xlim(xmin, xmax)
                    axs[row_idx][column_idx].set_ylim(ymin, ymax)


#################################################################
#################################################################
# This runs with the VIIRS points
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from copy import copy

# The path to the data folder
data_path = local_folder

# Get the list of dated subfolders
date_folders = [folder for folder in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, folder))]

# Sort the dated subfolders in ascending order
date_folders.sort()

# Ensure 'time' is in datetime format
gdf_viirs_perim['time'] = pd.to_datetime(gdf_viirs_perim['time'])

# Get the unique dates from the gdf_viirs_perim
# Extract date and hour from 'time' column
gdf_viirs_perim['date'] = gdf_viirs_perim['time'].dt.date.astype(str)
gdf_viirs_perim['hour'] = gdf_viirs_perim['time'].dt.hour.astype(str)

# Get unique dates and hours
unique_dates = gdf_viirs_perim['date'].unique()
unique_hours = gdf_viirs_perim['hour'].unique()

unique_dates = unique_dates.astype(str)

# Combine the dates from the gdf_viirs_perim and the subfolders
#all_dates = np.unique(np.concatenate((date_folders, unique_dates)))
all_dates = np.unique(np.concatenate((date_folders, unique_dates.astype(str))))

print(all_dates)

# Filter dates based on range
end_date = ed
end_date = datetime.strptime(end_date, '%Y-%m-%d').date()

# Filter dates based on range
if isinstance(start_date, str):
  start_date = datetime.strptime(start_date, '%Y-%m-%d')

print('start: ', start_date)
print('end: ', end_date)

filtered_dates = [date for date in all_dates if start_date <= datetime.strptime(date, '%Y-%m-%d').date() <= end_date]

# Do something elseend_date = datetime.strptime('2020-09-08', '%Y-%m-%d').date()
#filtered_dates = [date for date in all_dates if start_date <= datetime.strptime(date, '%Y-%m-%d').date() <= end_date]

# The rest of the code is identical until the loop where it checks if the date exists in the dataframe
# Create a grid of subplots

# Calculate the number of columns needed for the subplots
n_columns = len(filtered_dates)

# Define the buffer size (in meters)
buffer_size = 5000

# Calculate the bounds of the asp data
xmin, ymin, xmax, ymax = src.bounds

# Apply the buffer to the bounds
xmin -= buffer_size
ymin -= buffer_size
xmax += buffer_size
ymax += buffer_size

# Open the fire perimeter file
with rasterio.open(fire_fp) as fire_src:
    # Read the fire perimeter data
    fire = fire_src.read(1)

    # Mask no_data values
    nodata = fire_src.nodata
    fire = np.ma.masked_equal(fire, nodata)

#fig, axs = plt.subplots(2, n_columns, figsize=(20*n_columns, 16*3))

# Iterate over filtered dates
print('filtered dates')
print(filtered_dates)
for column_idx, date in enumerate(filtered_dates):
    print(date)
    # Set column titles
    axs[0, column_idx].set_title(date, fontsize=45, pad=20)

    # Check if 'date' is a datetime object, if so format it as a string
    if isinstance(date, datetime):
        date = date.date()

    # If the date exists in the gdf_viirs_perim
    if date in unique_dates:
        print(date)
        # Filter the gdf_viirs_perim for the current date
        gdf_date = gdf_viirs_perim[gdf_viirs_perim['date'] == date]

        print(gdf_date.head())
        # No need to transform coordinates, as gdf_viirs_perim is already in correct CRS
        gdf_date = gdf_date.to_crs("EPSG:32611")

        # Instead of plotting scatter points, plot the geometry directly
        # For each unique hour
        print('cur date: ', date)
        for row_idx, hour in enumerate(unique_hours):


            # Filter the gdf_date for the current hour
            gdf_time = gdf_date[gdf_date['hour'] == hour]

            # Convert gdf_time to correct CRS
            gdf_time = gdf_time.to_crs("EPSG:32611")

            # Instead of scatter plot, plot the geometry directly
            #asp_downsampled = asp[::10, ::10]  # take every 10th data point in each direction
            axs[row_idx][column_idx].imshow(asp_downsampled, cmap='gray', extent=(xmin, xmax, ymin, ymax))

            start_time = '0000'
            end_date = date
            end_time = str(copy(hour)) +'00'
            print(start_date, start_time, end_date, end_time)

            max_val = count_seconds(start_date, start_time, end_date, end_time)
            print('hour: ', hour)
            print('Current Seconds: ', max_val )
            fire_masked = np.ma.masked_where(fire > max_val, fire)
            axs[row_idx][column_idx].imshow(fire_masked, cmap='viridis', extent=(xmin, xmax, ymin, ymax))

            gdf_time.plot(ax=axs[row_idx][column_idx], color='red', alpha=0.5)

            # Set the title for the subplot to the hour
            axs[row_idx][column_idx].set_title(f"VIIRS Acquisition Time: {hour}", fontsize=36)

# After the creation of the subplots add these lines for column titles:
for ax, col in zip(axs[0], filtered_dates):
    ax.annotate(col, xy=(0.5, 1), xytext=(0, 40),
                xycoords='axes fraction', textcoords='offset points',
                ha='center', va='baseline', fontsize=40)

# And these lines for the row titles:
for ax, row in zip(axs[:,0], range(1, 3)):
    ax.annotate(f'VIIRS Overpass # {row}', xy=(0, .5), xytext=(-ax.yaxis.labelpad - 5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                ha='right', va='center', fontsize=40)


fig.suptitle('ELMFIRE Model Simulations overlayed with available VIIRS + NIROPS data + Chen et al (2020) VIIRS fire perimeters', fontsize=85, y=-0.03)
import matplotlib.lines as mlines

# Create the artists for the legend
# Create the artists for the legend with thicker lines
nirops_artist = mlines.Line2D([], [], color='blue', marker='_', markersize=15, linewidth=20, label='NIROPS')
viirs_perimeter_artist = mlines.Line2D([], [], color='red', marker='_', markersize=15, linewidth=20, label='VIIRS fire perimeter from (Chen et al. 2020)')
viirs_frp_artist = mlines.Line2D([], [], color='black', linestyle='None', marker='.', markersize=60, label='raw VIIRS FRP')
fire_perimeter_artist = mlines.Line2D([], [], color='green', marker='_', markersize=15, linewidth=20, label='fire perimeter')

# Create a list of the artists
artists = [nirops_artist, viirs_perimeter_artist, viirs_frp_artist, fire_perimeter_artist]

# Create the legend
plt.figlegend(handles=artists, loc='lower center', ncol=2, fontsize=100, bbox_to_anchor=(0.5, -0.3))

# Show the plot
plt.tight_layout()
plt.show()

# Save the figure
fig.savefig(os.path.join(directory, 'spread_plot.png'))

